In [ ]:
# Mount google drive and move to the project directory
from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/MyDrive/my_icdar

In [1]:
import os
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
from tkinter.filedialog import askopenfilename, askdirectory
try:
    from patchify import patchify
except:
    %pip install patchify
    from patchify import patchify
    
from utils import center_crop, rebuild_from_patches

In [2]:
# Load the input and ground truth images 
#d1 = askdirectory()
x = askopenfilename(initialdir='.', title='Select input image from validation data')
y = askopenfilename(title='Select output image from validation data')
x_initial_valid = plt.imread(x)
y_initial_valid = plt.imread(y)

CROP_SIZE_W = 3072 
CROP_SIZE_H = 3072

x_initial_valid = center_crop(x_initial_valid, (CROP_SIZE_H, CROP_SIZE_W))
y_initial_valid = center_crop(y_initial_valid, (CROP_SIZE_H, CROP_SIZE_W))

In [3]:
INPUT_SHAPE = (512, 512, 3)
PATCHES_STEP = 256

input_patches = np.squeeze(patchify(x_initial_valid, INPUT_SHAPE, step=PATCHES_STEP))
ground_truth_patches = np.squeeze(patchify(y_initial_valid, INPUT_SHAPE[:-1], step=PATCHES_STEP))

x = np.reshape(input_patches, ((-1, ) + input_patches.shape[-3:]))
y = np.reshape(ground_truth_patches, ((-1, ) + ground_truth_patches.shape[-2:]))
patches_num = x.shape[0]

In [4]:
# Display a list with the available models and ask the user to choose which to use
models_path = askdirectory(initialdir='.', title='Select the directory with the models' )
models_list = [name for name in os.listdir(models_path) if os.path.isdir(os.path.join(models_path, name))]
if 'logs' in models_list:
    models_list.remove('logs')

bce = keras.losses.BinaryCrossentropy()

predictions = []
for modelname in models_list:
    print(f'Loading ==> {modelname}')
    model = keras.models.load_model(os.path.join(models_path, modelname))
    splitted_modelname = modelname.split('_')[3:6]
    prediction = np.squeeze(model.predict(x))
    binarized_prediction = np.where(prediction > 0.5, 1, 0)
    loss = np.zeros([patches_num, 1])
    mean_iou = np.zeros([patches_num, 1])
    for i in range(patches_num):
        m = keras.metrics.MeanIoU(num_classes=2) # NOTE: To check why it gives different results when is outside loop
        loss[i] = bce(y[i], prediction[i]).numpy()
        m.update_state(y[i], binarized_prediction[i])
        mean_iou[i] = m.result().numpy()
            
    d = {
        'mode': splitted_modelname[1] + ' ' + splitted_modelname[2],
        'initial images': splitted_modelname[0].replace('images', ''),
        'model output': binarized_prediction,
        'loss': loss,
        'mean_iou': mean_iou
    }
    predictions.append(d)

Loading ==> unet_input512_filters32_images160_independent_patches_optAdam_lr0.0010_eps150
4/4 [==============================] - 6s 789ms/step
Loading ==> unet_input512_filters32_images160_random_patches_optAdam_lr0.0010_eps150
4/4 [==============================] - 1s 237ms/step
Loading ==> unet_input512_filters32_images589_overlapped_patches_optAdam_lr0.0010_eps150
4/4 [==============================] - 1s 287ms/step
Loading ==> unet_input512_filters32_images589_random_patches_optAdam_lr0.0010_eps150
4/4 [==============================] - 1s 253ms/step


In [5]:
np.save(os.path.join(models_path, 'predictions.npy'), predictions)

In [6]:
validation_pairs = {'x': x, 'y': y}
np.save(os.path.join(models_path, 'validation_pairs.npy'), validation_pairs)